In [1]:
import os
import pandas as pd
import pandas as pd
import numpy as np
from keras.layers.merge import concatenate
from keras.layers import Dense, Input, Dropout, Bidirectional, LSTM, Embedding
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
import re
from sklearn import preprocessing
import glob
from sklearn.model_selection import train_test_split
import h5py
import statistics

Using TensorFlow backend.


In [2]:
os.listdir()

['test_tOlRoBf.csv',
 'SentimentAnalysis-Copy1.ipynb',
 'SentimentAnalysis.ipynb',
 '.ipynb_checkpoints',
 'submission.csv',
 'train_F3WbcTw.csv']

#### Reading Data Frame

In [3]:
df = pd.read_csv('train_F3WbcTw.csv')

In [4]:
df.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [5]:
print('Shape of Data Frame is ', df.shape)

Shape of Data Frame is  (5279, 4)


In [6]:
df.sentiment.value_counts()

2    3825
1     837
0     617
Name: sentiment, dtype: int64

In [7]:
reviews_train = list(df['text'])

In [8]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
# reviews_test_clean = preprocess_reviews(reviews_test)

In [37]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words = remove_stop_words(reviews_train_clean)

In [10]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews = get_stemmed_text(reviews_train_clean)

In [21]:
# no_stop_words = remove_stop_words(stemmed_reviews)

In [9]:
def get_lemmatized_text(corpus):
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews = get_lemmatized_text(reviews_train_clean)

In [10]:
def clean_sentence(sentence):
        text = str(sentence)
        text = text.replace('.', " . ")
        text = text.replace('"', '')
        text = text.replace('”', '')
        text = text.replace('“', '')
        text = text.replace('%', ' % ')
        text = text.replace('®', ' ® ')
        text = text.replace('$', ' $ ')
        text = text.replace("'", '')
        text = text.replace('/', " / ")
        text = text.replace('-', ' - ')
        text = text.replace('&', ' & ')
        text = text.replace('?', " ? ")
        text = text.replace(',', " , ")
        text = text.replace(':', " : ")
        text = text.replace(';', " ; ")
        text = text.replace('(', " ( ")
        text = text.replace(')', " ) ")
        text = text.replace('[', " [ ")
        text = text.replace(']', " ] ")
#         text = text.replace('_',' _ ')
        text = text.replace('+',' + ')
        text = text.replace('⃝','')
        # text = text.replace('\uf0a3', "")
        # text = text.replace('\uf072','')
        text = text.replace('…',' … ')
        # text = text.replace('\uf06d','')
        text = text.replace(' ','')
        text = text.replace('','')
        # text = text.replace('\uf0a4','')
        text = text.replace('','')
        # text = text.replace('\uf052','')
        text = text.replace('\t', " ")
        text = text.replace('\n', " ")

        return text

In [57]:
reviews_train_clean[0]

'autoimmune diseases tend to come in clusters as for gilenya – if you feel good don’t think about it it won’t change anything but waste your time and energy i’m taking tysabri and feel amazing no symptoms other than dodgy color vision but i’ve had it since always so don’t know and i don’t know if it will last a month a year a decade ive just decided to enjoy the ride no point in worrying'

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
target = y
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
# Accuracy for C=0.01: 0.88416
# Accuracy for C=0.05: 0.892
# Accuracy for C=0.25: 0.89424
# Accuracy for C=0.5: 0.89456
# Accuracy for C=1: 0.8944
    
final_ngram = LogisticRegression(C=0.5)
final_ngram.fit(X, target)

/home/ai/anaconda3/envs/corgen/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/ai/anaconda3/envs/corgen/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ai/anaconda3/envs/corgen/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy for C=0.01: 0.7143939393939394
Accuracy for C=0.05: 0.7212121212121212
Accuracy for C=0.25: 0.7196969696969697
Accuracy for C=0.5: 0.7212121212121212
Accuracy for C=1: 0.7196969696969697


LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
# ngram_vectorizer.fit(reviews_train_clean)
# X = ngram_vectorizer.transform(reviews_train_clean)
# # X_test = ngram_vectorizer.transform(reviews_test_clean)
# target = y
# X_train, X_val, y_train, y_val = train_test_split(
#     X, target, train_size = 0.90
# )

# for c in [0.001,0.01, 0.05, 0.25, 0.5, 1]:
    
#     svm = LinearSVC(C=c)
#     svm.fit(X_train, y_train)
#     print ("Accuracy for C=%s: %s" 
#            % (c, accuracy_score(y_val, svm.predict(X_val))))
    
# # Accuracy for C=0.01: 0.89104
# # Accuracy for C=0.05: 0.88736
# # Accuracy for C=0.25: 0.8856
# # Accuracy for C=0.5: 0.88608
# # Accuracy for C=1: 0.88592
    
# final_svm_ngram = LinearSVC(C=0.01)
# final_svm_ngram.fit(X, target)
# # print ("Final Accuracy: %s" 
# #        % accuracy_score(target, final_svm_ngram.predict(X_test)))

In [11]:
stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(lemmatized_reviews)
X = ngram_vectorizer.transform(lemmatized_reviews)
# X_test = ngram_vectorizer.transform(reviews_test_clean)
target = df['sentiment'].astype(str)
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.9
)

for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

final = LinearSVC(C=0.01)
final.fit(X, target)
# print ("Final Accuracy: %s" 
#        % accuracy_score(target, final.predict(X_test)))


/home/ai/anaconda3/envs/corgen/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Accuracy for C=0.001: 0.7575757575757576
Accuracy for C=0.005: 0.7613636363636364
Accuracy for C=0.01: 0.7651515151515151


/home/ai/anaconda3/envs/corgen/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy for C=0.05: 0.7443181818181818
Accuracy for C=0.1: 0.7386363636363636


LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [ ]:
X_test = ngram_vectorizer.transform(reviews_test_clean)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

In [41]:
len_list = [len(clean_sentence(item).split()) for item in list(df['text'])]

In [42]:
statistics.median(len_list)

166

In [43]:
max_len = statistics.median(len_list)#max(len(clean_sentence(item).split()) for item in list(df['text']))

In [44]:
# clean sentences in text
df['text'] = df['text'].apply(clean_sentence)

In [45]:
df.columns

Index(['unique_hash', 'text', 'drug', 'sentiment'], dtype='object')

In [46]:
df['fulltext'] = no_stop_words #df['drug'] + ' ' + df['text']

In [47]:
# max_len = 80#int(np.mean(h))
print('preprocessing data...')
tokenizer = Tokenizer()#num_words= vocabulary_size

tokenizer.fit_on_texts(df['fulltext'])
vocab_size = len(tokenizer.word_index)+1
sentences = tokenizer.texts_to_sequences((df['fulltext']))
X_sentences = pad_sequences(sentences, maxlen=max_len, padding='post', truncating = 'post')

# X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1,shuffle=False)

preprocessing data...


In [15]:
y = df['sentiment'].astype(str)

In [49]:
le = preprocessing.LabelEncoder()
le.fit(y)
# # y_train = np.asarray(pd.get_dummies(y))
y = to_categorical(le.transform(y), num_classes=None)
list(le.classes_)

['0', '1', '2']

In [50]:
print('Shape of questions tensor:', X_sentences.shape)
#print('Shape of instructions tensor:', X_tags.shape)
print('Shape of label tensor:', y.shape)
num_classes = len(le.classes_)
print('num_classes : ', num_classes)

Shape of questions tensor: (5279, 166)
Shape of label tensor: (5279, 3)
num_classes :  3


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sentences, y, test_size=0.1, random_state=0)

In [52]:
import keras.backend as K
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

#### Test data predictions

In [12]:
test_df = pd.read_csv('test_tOlRoBf.csv')

In [13]:
test_df.head()

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara


In [14]:
test_df.shape

(2924, 3)

In [15]:
reviews_test = list(test_df['text'])
reviews_test_clean = preprocess_reviews(reviews_test)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)
X_test = ngram_vectorizer.transform(lemmatized_reviews_test)
preds = final.predict(X_test)
# print ("Final Accuracy: %s" % accuracy_score(target, final_ngram.predict(X_test)))
# no_stop_words_test = remove_stop_words(reviews_test_clean)

In [135]:
reviews_test = list(test_df['text'])
reviews_test_clean = preprocess_reviews(reviews_test)
no_stop_words_test = remove_stop_words(reviews_test_clean)
# lemmatized_reviews_test = get_s(no_stop_words_test)

In [136]:
test_df['fulltext'] = no_stop_words_test#test_df['drug'] + ' '+ test_df['text']
sentences_test = tokenizer.texts_to_sequences((test_df['fulltext']))
X_sentences_test = pad_sequences(sentences_test, maxlen=max_len, padding='post', truncating = 'post')

In [137]:
preds = model.predict(X_sentences_test)

In [16]:
label_list = list(preds)

In [138]:
label_list = [le.inverse_transform([np.argmax(pred, axis = -1)])[0] for pred in preds]

In [35]:
# for _ind in range(len(test_df)):
#     p = np.argmax(pred, axis = -1)
#     sentiment = le.inverse_transform([p[0]])
#     test_df.loc[_ind, 'sentiment'] = sentiment[0]

In [17]:
test_df['sentiment'] = label_list

In [18]:
test_df.head()

,unique_hash,text,drug,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod,2
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod,2
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara,2


In [19]:
test_df.columns

Index(['unique_hash', 'text', 'drug', 'sentiment'], dtype='object')

In [20]:
test_df.sentiment.value_counts()

2    2716
1     131
0      77
Name: sentiment, dtype: int64

In [21]:
final_df = test_df[['unique_hash', 'sentiment']]

In [22]:
final_df.to_csv('submission.csv', index= None)